### Neural Network for Regression (Exercise)

This dataset encompasses details about various workers and their corresponding employment levels, featuring a diverse set of attributes ranging from categorical to continuous. Initialize the data loading process using the suitable Pandas function, and meticulously inspect for any instances of null or duplicated data. Specifically focusing on the **“salary in usd”** feature, identify and eliminate outliers while devising a strategy to address any missing values.

Then, use any method you like to encode the categorical features, namely **“work year”, “experience level”, “employment type”, “job title”, “employee residence”, “remote ratio”, “company location”, and “company size”**. You may consider to employ the sklearn LabelEncoder class<sup>1</sup>.

Following the preprocessing steps, normalize the dataset utilizing the z-score technique to ensure consistent scaling across features. Subsequently, construct a neural network using **PyTorch**, incorporating **2 hidden layers with 5 and 3 neurons**, respectively. Carefully select an appropriate learning rate and normalization value for optimal model training.

Furthermore, assess the model’s performance using a relevant evaluation metric, ensuring a comprehensive understanding of its effectiveness in handling the given employment dataset. Finally, find the best hyperparameter combination (namely **lr** and **weight decay**) using both the **Grid Search** and the **k-fold cross validation** methods.

<sup>1</sup>https://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.LabelEncoder.html

---

**First Version**

Here, I first present a simpler implementation that strictly follows the track's instructions and utilizes Sklearn's GridSearchCV.

In [45]:
import pandas as pd
import numpy as np
import itertools

import torch
from torch import nn
from torch import optim

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from category_encoders import TargetEncoder
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.base import BaseEstimator, TransformerMixin

In [46]:
# Load Dataset
df = pd.read_csv('./datasets/ds_salaries.csv')
# Here I choose to delete some redundant features such as salary and salary_currency since thay are too similar to our target variable 'salary_in_usd'
df.drop(['salary', 'salary_currency'], axis=1, inplace=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

display(df)

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Machine Learning Software Engineer,168000,CA,100,CA,M
1,2023,SE,FT,Data Analyst,179975,US,100,US,M
2,2022,SE,FT,Data Scientist,144000,US,100,US,M
3,2023,SE,FT,Applied Scientist,222200,US,0,US,L
4,2021,EX,FT,Head of Data,230000,RU,50,RU,L
...,...,...,...,...,...,...,...,...,...
3750,2023,SE,FT,Machine Learning Engineer,150000,US,100,US,M
3751,2023,SE,FT,Data Analyst,180180,US,0,US,M
3752,2023,EX,FT,Data Engineer,310000,US,100,US,M
3753,2021,MI,FT,Research Scientist,62649,FR,50,FR,M


In [47]:
# Check for missing values
print("\nMissing Values:", df.isnull().sum().sum())
# Check for duplicates
print("\nDuplicates:", df.duplicated().sum())


Missing Values: 0

Duplicates: 1171


In [ ]:
# In this case we found that we don't have missing values. However in general if would have some of them we could handle it as follows:
# - for a Numerical feature we could think to use the median of its values
# - for a Categorical feature we could think to use the most frequent value (i.e. the mode of its values)

# For what regard the duplicates we have 1171 duplicates ... we can drop them
df.drop_duplicates(inplace=True)
print(df.shape)

(2584, 9)


In [49]:
# Identify and remove outliers in "salary in usd" using IQR
Q1 = df['salary_in_usd'].quantile(0.25)
Q3 = df['salary_in_usd'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# we take just entries within the range [lower_bound, upper_bound]
df = df[(df['salary_in_usd'] >= lower_bound) & (df['salary_in_usd'] <= upper_bound)]

print(df.shape)

(2555, 9)


**Encoding of Categorical Features**

To apply **standardization**, all features must be numeric. While continuous features are already numerical, categorical features must first be **transformed into numerical values**. This transformation process is called **encoding**. One way we can encode categorical features is using LabelEncoder.

In [ ]:
numerical_features = df.select_dtypes(include=['float64', 'int64']).columns
categorical_features = df.select_dtypes(exclude=['float64', 'int64']).columns

print(numerical_features)
print(categorical_features)

# Encoding of the categorical features
le = LabelEncoder()
for cat in categorical_features:
    df[cat] = le.fit_transform(df[cat])

Index(['work_year', 'salary_in_usd', 'remote_ratio'], dtype='object')
Index(['experience_level', 'employment_type', 'job_title',
       'employee_residence', 'company_location', 'company_size'],
      dtype='object')


In [51]:
X = df.drop('salary_in_usd', axis=1).values
y = df['salary_in_usd'].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalization with z-score using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [52]:
class RegressionNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RegressionNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size[0]) # Input -> First Hidden Layer
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1]) # First Hidden Layer layer -> Second Hidden Layer
        self.fc3 = nn.Linear(hidden_size[1], output_size) # Second Hidden Layer -> Output Layer
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x) # In regression task the sigmoid is not applied at the output layer
        return x

When using Scikit-Learn’s `GridSearchCV`, we cannot directly pass a PyTorch model as it does not conform to Scikit-Learn’s standardized estimator interface. `GridSearchCV` expects an estimator that implements  at least the `fit` and `predict` methods following the Scikit-Learn API.
To make our PyTorch neural network compatible with Scikit-Learn, we need to create a wrapper class, `PytorchNN`, which encapsulates our model and inherits from `BaseEstimator` and `TransformerMixin`.

In [ ]:
class PytorchNN(BaseEstimator, TransformerMixin):
    def __init__(self, input_size, hidden_size, output_size, lr=0.1, weight_decay=0.001, epochs=1000):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lr = lr
        self.weight_decay = weight_decay
        self.epochs = epochs
        self.simple_nn = RegressionNN(self.input_size, self.hidden_size, self.output_size)

    def fit(self, X, y):
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

        criterion = nn.MSELoss()
        optimizer = optim.SGD(self.simple_nn.parameters(), lr=self.lr, weight_decay=self.weight_decay)

        for epoch in range(self.epochs):
            self.simple_nn.train()
            optimizer.zero_grad()
            output = self.simple_nn(X_tensor)
            loss = criterion(output, y_tensor)
            loss.backward()
            optimizer.step()
        return self

    def predict(self, X):
        self.simple_nn.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32)
        with torch.no_grad():
            output = self.simple_nn(X_tensor)
        return output

In this way we can easily integrate the neural network within Scikit-Learn workflows, enabling us to use `GridSearchCV` for hyperparameter optimization.

Another important point to note is that Scikit-Learn's GridSearchCV operates by **maximizing** the metric specified in the `scoring` parameter. However, in regression tasks, our goal is to minimize the Mean Squared Error (MSE). To align with Scikit-Learn's implementation, we should use `neg_mean_squared_error`, meaning we **maximize the negative MSE** instead. This ensures proper optimization during hyperparameter tuning. You can find a full list of available evaluation metrics at [Model Evaluation - Scoring Parameter](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter).

In [54]:
# Define hyperparameter grid
param_grid = {
    'lr': [0.1, 0.01, 0.001],
    'weight_decay': [0.0, 0.1, 0.01]
}

model = PytorchNN(input_size=X.shape[1], hidden_size=[5, 3], output_size=1)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=True)
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

# Run the grid search
grid_search.fit(X_train, y_train)

# Print the best score and best params
print("\nBest Score: \n", - grid_search.best_score_)
print("\nBest Hyperparameters: \n", grid_search.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits

Best Score: 
 20809608240.26951

Best Hyperparameters: 
 {'lr': 0.1, 'weight_decay': 0.0}


In [55]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print(y_pred[:10])

tensor([[404.2125],
        [404.2125],
        [404.2125],
        [404.2125],
        [404.2125],
        [404.2125],
        [404.2125],
        [404.2125],
        [404.2125],
        [404.2124]])


---

---

**Second Version**

Now, let's explore a different implementation that highlights other aspects, such as best practices for encoding. Additionally, since Scikit-Learn only supports CPU, while PyTorch can leverage both CPU and GPU, it is often more efficient to handle everything directly in PyTorch. Therefore, we will also implement GridSearch with K-Fold cross-validation from **scratch**.

**Encoding of Categorical Features**

Above we used the Label Encoder to encode all categorical variables as the track recommended, but I can tell you that it's not a good move. LabelEncoder is designed to encode target labels with value between 0 and n_classes-1, therefore it make no sense to use it for features encoding. Moreover, when encoding features is important to understand the typology of feature considered, since different typologies lead to [different encodings](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features).

Specifically, categorical features fall into two main categories:
1. **Ordinal Features**: Have a meaningful order (e.g., education level: *High School < Bachelor's < Master's < PhD*).
2. **Nominal Features**: Have no intrinsic order (e.g.company location).

**1️⃣ Ordinal Features → Ordinal Encoding**

Assigns numerical values based on category order. For example, education levels may be assigned values such as:
- High School → 0
- Bachelor's → 1
- Master's → 2
- PhD → 3

Since ordinal variables have a meaningful ranking, converting them into ordered numbers preserves that relationship.

**2️⃣ Nominal Features → One-Hot Encoding (OHE)**

Creates binary columns for each category. For instance, if a feature represents company size with values **S, M, L**, one-hot encoding transforms it into three separate columns:
- `S` → [1, 0, 0]
- `M` → [0, 1, 0]
- `L` → [0, 0, 1]

One-hot encoding ensures that categories are represented equally without implying any order.

⚠️ **Issue: High Cardinality in Nominal Features**
For categorical features with many unique values (e.g., thousands of job titles), one-hot encoding creates too many columns, which could lead to the curse of dimensionality.
A better alternative in such a case is **Target Encoding**.

**3️⃣ Alternative for High-Cardinality Nominal Features → Target Encoding**

Replaces each category with the **mean target value** (e.g., average salary for each job title).
- Example: If the average salary for “Software Engineer” is 80,000 USD and for “Data Scientist” is 90,000 USD, we replace:
Software Engineer → 80,000
Data Scientist → 90,000

It solve the one-hot encoding **dimensionality explosion** problem (i.e. when dealing with features that have a large number of unique values) while keeping useful statistical information about the relationship between a category and the target variable.

##### **When to Apply Encoding?**
Just like standardization, encoding should be computed only on the training set:
- Fit and transform on the training data (fit_transform).
- Transform only on the test set (transform).

⚠️ In general, always rember that the test set should remain **untouched**, mimicking real-world data the model has never seen before.

Once all features are numeric (including encoded categorical ones), we can apply Z-score standardization to all features (both the transformed categorical ones and the original continuous numerical features).

This ensures that all features are on the same scale, having a mean of 0 and a standard deviation of 1, helping models like neural networks learn efficiently.

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [44]:
df = pd.read_csv('./datasets/ds_salaries.csv')
df.drop(['salary', 'salary_currency'], axis=1, inplace=True)

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Check for missing values -> NO missing values
print("\nMissing Values:", df.isnull().sum().sum())
# Check for duplicates
print("\nDuplicates:", df.duplicated().sum())

# Drop duplicates
df = df.drop_duplicates()

# Identify and remove outliers in "salary in usd" using IQR
alpha = 1.5
Q1 = df['salary_in_usd'].quantile(0.25)
Q3 = df['salary_in_usd'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - alpha * IQR
upper_bound = Q3 + alpha * IQR

# we take just entries within the range [lower_bound, upper_bound]
df = df[(df['salary_in_usd'] >= lower_bound) & (df['salary_in_usd'] <= upper_bound)]

print("\n",df.shape)


Missing Values: 0

Duplicates: 1171

 (2555, 9)


In [18]:
X = df.drop('salary_in_usd', axis=1)
y = df['salary_in_usd']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

To handle categorical features we will proceed as follows: we first distinguish between ordinal and nominal features. 

For the **ordinal features** we use a Ordinal Encoder, instead for the **nominal features** if the number of unique values is fewer than 10 we apply **One-Hot Encoding**, otherwise we apply **Target Encoding** (to reduce dimensionality explosion).

In [19]:
ordinal_features = ['experience_level', 'company_size']
nominal_features = ['employment_type', 'job_title', 'employee_residence', 'company_location']

one_hot_features = []  # To be filled dynamically
target_features = []   # To be filled dynamically 

# For ordinal features, you can simply use OrdinalEncoder
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_train[ordinal_features] = ordinal_encoder.fit_transform(X_train[ordinal_features])
X_test[ordinal_features] = ordinal_encoder.transform(X_test[ordinal_features])


threshold = 10 # max unique values
# Let's handle nominal features features
for col in nominal_features:
    if X_train[col].nunique() <= threshold:
        one_hot_features.append(col)  # If number of unique values is below threshold, use one-hot encoding
    else:
        target_features.append(col)  # Otherwise, use target encoding
# Note: Basically here we will one_hot_features and target_features based on the number of unique values

print("One-hot encoding features: ", one_hot_features)
print("Target encoding features: ", target_features)

# Apply One-Hot Encoding for the `one_hot_features`
onehot_encoder = OneHotEncoder(sparse_output=False, dtype=float, handle_unknown='ignore')
X_train_onehot = onehot_encoder.fit_transform(X_train[one_hot_features])
X_test_onehot = onehot_encoder.transform(X_test[one_hot_features])
X_train_onehot_df = pd.DataFrame(X_train_onehot, 
                                 columns=onehot_encoder.get_feature_names_out(one_hot_features), 
                                 index=X_train.index)
X_test_onehot_df = pd.DataFrame(X_test_onehot, 
                                columns=onehot_encoder.get_feature_names_out(one_hot_features), 
                                index=X_test.index)
X_train = pd.concat([X_train, X_train_onehot_df], axis=1).drop(columns=one_hot_features)
X_test = pd.concat([X_test, X_test_onehot_df], axis=1).drop(columns=one_hot_features)

# Apply Target Encoding for the `target_features`
target_encoder = TargetEncoder()
X_train[target_features] = target_encoder.fit_transform(X_train[target_features], y_train)
X_test[target_features] = target_encoder.transform(X_test[target_features])

One-hot encoding features:  ['employment_type']
Target encoding features:  ['job_title', 'employee_residence', 'company_location']


In [20]:
display(X_train)

,work_year,experience_level,job_title,employee_residence,remote_ratio,company_location,company_size,employment_type_CT,employment_type_FL,employment_type_FT,employment_type_PT
3749,2023,3.0,138032.720430,150378.151839,0,148754.081258,1.0,0.0,0.0,1.0,0.0
3551,2021,0.0,115766.207915,41918.604303,100,38347.912811,1.0,0.0,0.0,1.0,0.0
2617,2023,3.0,163090.452572,150378.151839,100,148754.081258,1.0,0.0,0.0,1.0,0.0
2596,2023,3.0,104597.610592,150378.151839,100,148754.081258,1.0,0.0,0.0,1.0,0.0
3345,2022,1.0,172206.957588,150378.151839,0,148754.081258,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2156,2023,3.0,130669.230047,150378.151839,0,148754.081258,1.0,0.0,0.0,1.0,0.0
1325,2022,2.0,130669.230047,150378.151839,0,148754.081258,1.0,0.0,0.0,1.0,0.0
1378,2023,2.0,142020.375994,150378.151839,0,148754.081258,1.0,0.0,0.0,1.0,0.0
1623,2022,3.0,104597.610592,150378.151839,100,148754.081258,1.0,0.0,0.0,1.0,0.0


In [21]:
# transform in numpy array
X_train = X_train.values
y_train = y_train.values
X_test = X_test.values
y_test = y_test.values

In [22]:
# Okay, now all features present numeric values, so as next step we can apply standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
display(X_train)

array([[ 0.93128887,  0.66335   ,  0.3671331 , ..., -0.05425922,
         0.11127609, -0.08000492],
       [-1.72545146, -2.44886284, -1.02700537, ..., -0.05425922,
         0.11127609, -0.08000492],
       [ 0.93128887,  0.66335   ,  1.93603386, ..., -0.05425922,
         0.11127609, -0.08000492],
       ...,
       [ 0.93128887, -0.37405428,  0.61680597, ..., -0.05425922,
         0.11127609, -0.08000492],
       [-0.3970813 ,  0.66335   , -1.72628736, ..., -0.05425922,
         0.11127609, -0.08000492],
       [-0.3970813 , -0.37405428, -0.17587874, ..., -0.05425922,
         0.11127609, -0.08000492]], shape=(2044, 11))

In [24]:
print(y_train.shape)

(2044,)


In [25]:
# Let's transform y_train and y_test to be column vectors (so they will match output layer of our NN)
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print(y_train.shape)

(2044, 1)


In [26]:
display(y_train)

array([[192000],
       [ 12171],
       [168100],
       ...,
       [230000],
       [135000],
       [ 75000]], shape=(2044, 1))

In [27]:
# Convert data to torch tensors
X_train_tensor = torch.from_numpy(X_train).float().to(device)
X_test_tensor = torch.from_numpy(X_test).float().to(device)
y_train_tensor = torch.from_numpy(y_train).float().to(device)
y_test_tensor= torch.from_numpy(y_test).float().to(device)

In [28]:
print(X_train_tensor)

tensor([[ 0.9313,  0.6633,  0.3671,  ..., -0.0543,  0.1113, -0.0800],
        [-1.7255, -2.4489, -1.0270,  ..., -0.0543,  0.1113, -0.0800],
        [ 0.9313,  0.6633,  1.9360,  ..., -0.0543,  0.1113, -0.0800],
        ...,
        [ 0.9313, -0.3741,  0.6168,  ..., -0.0543,  0.1113, -0.0800],
        [-0.3971,  0.6633, -1.7263,  ..., -0.0543,  0.1113, -0.0800],
        [-0.3971, -0.3741, -0.1759,  ..., -0.0543,  0.1113, -0.0800]],
       device='cuda:0')


In [29]:
class RegressionNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RegressionNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size[0]) # Input -> First Hidden Layer
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1]) # First Hidden Layer layer -> Second Hidden Layer
        self.fc3 = nn.Linear(hidden_size[1], output_size) # Second Hidden Layer -> Output Layer
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x) # In regression task the sigmoid is not applied at the output layer
        return x

In [30]:
torch.manual_seed(42)

# Initialize the model
input_size = X_train.shape[1]
hidden_size = [5, 3]
output_size = 1
model = RegressionNN(input_size, hidden_size, output_size).to(device)

# Loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.SGD(model.parameters(), lr=0.1)  # SGD optimizer


# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    
    # Forward pass
    preds_train_tensor = model(X_train_tensor)
    loss = criterion(preds_train_tensor, y_train_tensor)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if (epoch+1) % 100 == 0:
        # Evaluate the model on the test data
        model.eval()
        with torch.no_grad():
            # Compute MSE and MAE for test data (just for logging)
            preds_test_tensor = model(X_test_tensor)
            test_mse = criterion(preds_test_tensor, y_test_tensor)
            test_mae = mean_absolute_error(preds_test_tensor.cpu(), y_test_tensor.cpu()) # here I switch to CPU just because MAE of sklearn is not able to work on GPU

        print(f"Epoch: {epoch+1} | Training loss (MSE): {loss.item():.2f} | Test MSE: {test_mse.item():.2f}, Test MAE: {test_mae:.2f}")


Epoch: 100 | Training loss (MSE): 3886808064.00 | Test MSE: 4074648832.00, Test MAE: 52242.20
Epoch: 200 | Training loss (MSE): 3886808064.00 | Test MSE: 4074648832.00, Test MAE: 52242.20
Epoch: 300 | Training loss (MSE): 3886808064.00 | Test MSE: 4074648832.00, Test MAE: 52242.20
Epoch: 400 | Training loss (MSE): 3886808064.00 | Test MSE: 4074648832.00, Test MAE: 52242.20
Epoch: 500 | Training loss (MSE): 3886808064.00 | Test MSE: 4074648832.00, Test MAE: 52242.20
Epoch: 600 | Training loss (MSE): 3886808064.00 | Test MSE: 4074648832.00, Test MAE: 52242.20
Epoch: 700 | Training loss (MSE): 3886808064.00 | Test MSE: 4074648832.00, Test MAE: 52242.20
Epoch: 800 | Training loss (MSE): 3886808064.00 | Test MSE: 4074648832.00, Test MAE: 52242.20
Epoch: 900 | Training loss (MSE): 3886808064.00 | Test MSE: 4074648832.00, Test MAE: 52242.20
Epoch: 1000 | Training loss (MSE): 3886808064.00 | Test MSE: 4074648832.00, Test MAE: 52242.20


Since we want to perform **hyperparameter tuning** using **Grid Search** with **k-fold cross validation** it's efficient to encapsulate the entire process in a single method. This approach allows us to easily repeat the training and evaluation process as needed. Let's create a method to handle this.

In [31]:
def train_and_evaluate(model, criterion, optimizer, X_train_fold_tensor, y_train_fold_tensor, X_val_fold_tensor, y_val_fold_tensor, fold, num_epochs=1000):
    """
    Train and evaluate a PyTorch model.

    Parameters:
        model (torch.nn.Module): The neural network model.
        criterion (torch.nn.Module): The loss function (e.g., nn.MSELoss).
        optimizer (torch.optim.Optimizer): Optimizer for training (e.g., Adam, SGD).
        X_train_fold_tensor (torch.Tensor): Training features.
        y_train_fold_tensor (torch.Tensor): Training target values.
        X_val_fold_tensor (torch.Tensor): Test features.
        y_val_fold_tensor (torch.Tensor): Test target values.
        fold: Fold number considered (used just for logging)
        num_epochs (int): Number of training epochs.

    Returns:
        float: Final val loss (MSE).
    """

    for epoch in range(num_epochs):   
        model.train()
        # Forward pass
        preds_train = model(X_train_fold_tensor)
        loss = criterion(preds_train, y_train_fold_tensor)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
        if (epoch+1) % num_epochs == 0: # we print just the last step results
            # Evaluate the model on the test data
            model.eval()
            with torch.no_grad():
                # Compute MSE and MAE for validation data (just for logging)
                preds_val = model(X_val_fold_tensor)      
                val_mse = criterion(preds_val, y_val_fold_tensor)
                val_mae = mean_absolute_error(preds_val.cpu(), y_val_fold_tensor.cpu())
            print(f"  Fold: {fold} | Val loss (MSE): {val_mse.item():.2f}, Val MAE: {val_mae:.2f}")

    return val_mse.item()

Okay, now we are ready to actually perform hyperparameter tuning ...

In [32]:
torch.manual_seed(42)

cv_folds = 5 # Number of folds

# Define 5-fold cross-validation
kf = KFold(n_splits=cv_folds, shuffle=True, random_state=42)

best_params = None
best_loss = float('inf')
best_model_state = None

# Define hyperparameter grid
param_grid = {
    'lr': [0.1, 0.01, 0.001],
    'weight_decay': [0, 0.1, 0.01]
}

# Convert param_grid to all combinations of hyperparameters
param_combinations = list(itertools.product(*param_grid.values()))
num_candidates = len(param_combinations)  # Total hyperparameter combinations
total_fits = num_candidates * cv_folds  # Total model fits

print(f"Fitting {cv_folds} folds for each of {num_candidates} candidates, totalling {total_fits} fits.\n")
# Iterate over all parameter combinations
for idx, params in enumerate(param_combinations):
    lr, weight_decay = params
    print(f"\n[{idx+1}/{num_candidates}] Testing Params: lr={lr}, weight_decay={weight_decay}")

    fold_losses = []
    fold=0

    # Iteratively consider all fold configurations
    for train_index, val_index in kf.split(X_train):
        fold=fold +1
        # Split data into train and validation sets for the current fold
        X_train_fold = X_train[train_index]
        y_train_fold = y_train[train_index]
        X_val_fold = X_train[val_index]
        y_val_fold = y_train[val_index]

        # Convert data to torch tensors
        X_train_fold_tensor = torch.from_numpy(X_train_fold).float().to(device)
        y_train_fold_tensor = torch.from_numpy(y_train_fold).float().to(device)
        X_val_fold_tensor = torch.from_numpy(X_val_fold).float().to(device)
        y_val_fold_tensor = torch.from_numpy(y_val_fold).float().to(device)

        # Initialize model, optimizer (with the lr and weight decay values to test) and criterion
        model = RegressionNN(input_size=X_train_fold_tensor.shape[1], hidden_size=[5, 3], output_size=1).to(device)
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = nn.MSELoss()

        # Train and evaluate the model on the current fold
        fold_loss = train_and_evaluate(
            model=model,
            criterion=criterion,
            optimizer=optimizer,
            X_train_fold_tensor=X_train_fold_tensor,
            y_train_fold_tensor=y_train_fold_tensor,
            X_val_fold_tensor=X_val_fold_tensor,
            y_val_fold_tensor=y_val_fold_tensor,
            fold=fold
        )
        fold_losses.append(fold_loss)

    # Calculate average loss across all folds
    avg_loss = sum(fold_losses) / len(fold_losses)
    print(f"Avg Loss for params {params}: {avg_loss:.2f}")

    # Update the best parameters if the current configuration is better
    if avg_loss < best_loss:
        best_loss = avg_loss
        best_params = params
        best_model_state = model.state_dict() 

print('\n')
print(f"Best Params: lr={best_params[0]}, weight_decay={best_params[1]}")
print(f"Best Loss: {best_loss:.2f}")

Fitting 5 folds for each of 9 candidates, totalling 45 fits.


[1/9] Testing Params: lr=0.1, weight_decay=0
  Fold: 1 | Val loss (MSE): 4317376512.00, Val MAE: 53677.58
  Fold: 2 | Val loss (MSE): 3936137216.00, Val MAE: 51221.72
  Fold: 3 | Val loss (MSE): 3615389184.00, Val MAE: 48164.67
  Fold: 4 | Val loss (MSE): 4047974656.00, Val MAE: 50643.90
  Fold: 5 | Val loss (MSE): 3539492864.00, Val MAE: 48489.63
Avg Loss for params (0.1, 0): 3891274086.40

[2/9] Testing Params: lr=0.1, weight_decay=0.1
  Fold: 1 | Val loss (MSE): 4344276480.00, Val MAE: 53868.32
  Fold: 2 | Val loss (MSE): 3952835584.00, Val MAE: 51267.00
  Fold: 3 | Val loss (MSE): 3128572160.00, Val MAE: 44419.73
  Fold: 4 | Val loss (MSE): 4066266112.00, Val MAE: 50619.09
  Fold: 5 | Val loss (MSE): 3524785664.00, Val MAE: 48228.12
Avg Loss for params (0.1, 0.1): 3803347200.00

[3/9] Testing Params: lr=0.1, weight_decay=0.01
  Fold: 1 | Val loss (MSE): 4319204352.00, Val MAE: 53691.05
  Fold: 2 | Val loss (MSE): 393741

In [33]:
print(best_model_state)

OrderedDict({'fc1.weight': tensor([[ 2.7005e+05, -1.8811e+05, -4.1405e+05, -2.4953e+05,  5.7674e+03,
         -2.3878e+05,  3.2838e+05,  2.5743e+04,  2.5737e+04, -5.2785e+04,
          3.7949e+04],
        [-5.4235e+02,  5.5079e+03,  8.3578e+03,  5.0522e+03, -8.4918e+02,
          4.8349e+03, -5.0199e+03, -5.1970e+02, -5.1934e+02,  1.0653e+03,
         -7.6577e+02],
        [-2.2678e+03,  4.2710e+03, -1.1619e+04,  3.3713e+03,  6.7128e+03,
          3.2257e+03,  2.0274e+03, -3.4145e+02, -3.5254e+02,  7.1810e+02,
         -5.2058e+02],
        [-6.1468e+03,  5.7238e+04,  8.7687e+04,  5.9508e+04, -4.0708e+04,
          5.6942e+04, -1.2700e+04, -6.1380e+03, -6.1378e+03,  1.2588e+04,
         -9.0502e+03],
        [ 2.8840e+04, -1.6336e+04, -4.8248e+04, -3.2751e+04,  2.9720e+03,
         -3.1339e+04,  4.2363e+04,  3.3834e+03,  3.3760e+03, -6.9270e+03,
          4.9777e+03]], device='cuda:0'), 'fc1.bias': tensor([-474329.5000,    9571.5361,    6508.9658,  113119.8906,  -62217.1367],
       d

In [34]:
best_model = RegressionNN(input_size=X_test.shape[1], hidden_size=[5, 3], output_size=1).to(device)
best_model.load_state_dict(best_model_state)
best_model.eval()  # Set to evaluation mode

# Convert test data to tensor
X_test_tensor = torch.tensor(X_test, dtype=torch.float32, device=device)

# Make predictions
with torch.no_grad():
    y_pred_tensor = best_model(X_test_tensor)
    test_mse = criterion(y_pred_tensor, y_test_tensor)
    test_mae = mean_absolute_error(y_pred_tensor.cpu(), y_test_tensor.cpu())
print(f"Test loss (MSE): {test_mse.item():.2f}, Test MAE: {test_mae:.2f}")    

Test loss (MSE): 4087500544.00, Test MAE: 52371.81


In [35]:
print("Predictions on test set:", y_pred_tensor[:15])

Predictions on test set: tensor([[128070.1875],
        [128070.1875],
        [128070.1875],
        [128070.1875],
        [128070.1875],
        [128070.1875],
        [128070.1875],
        [128070.1875],
        [128070.1875],
        [128070.1875],
        [128070.1875],
        [128070.1875],
        [128070.1875],
        [128070.1875],
        [128070.1875]], device='cuda:0')


---

Looking our prediction results seems that we got stuck in a local minima ...

... as we already saw in scratch implementation of Neural Network for Regression when target variable has high magnitude values can have a more difficult training resulting in this type of problem

So, now let's try to repeat the same we did above but with normalized target variable ...

In [36]:
# Z-normalize also the target variable
target_scaler = StandardScaler()
y_train_std = target_scaler.fit_transform(y_train)
y_train_std = torch.from_numpy(y_train_std).float().to(device) # convert it into tensor
# we leave y-test as it is

In [37]:
display(y_train_std)

tensor([[ 0.9866],
        [-1.8979],
        [ 0.6032],
        ...,
        [ 1.5961],
        [ 0.0723],
        [-0.8901]], device='cuda:0')

In [38]:
torch.manual_seed(42)

# Initialize the model
model = RegressionNN(input_size, hidden_size, output_size).to(device)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    
    # Forward pass
    preds_train_std = model(X_train_tensor)
    loss = criterion(preds_train_std, y_train_std)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

# Prediction 
model.eval()
with torch.no_grad():
    preds_test_std = model(X_test_tensor)
    # De-normalized predictions (to do it we need first to detach and transform them in numpy arrays)
    preds_test_numpy = target_scaler.inverse_transform(preds_test_std.detach().cpu().numpy())
    preds_test = torch.from_numpy(preds_test_numpy).float().to(device) # convert back to tensor
    test_mse = criterion(preds_test, y_test_tensor)
    test_mae = mean_absolute_error(preds_test.cpu(), y_test_tensor.cpu())
print(f"Test loss (MSE): {test_mse.item():.2f}, Test MAE: {test_mae:.2f}")

Test loss (MSE): 2378938368.00, Test MAE: 37803.72


In [39]:
print(preds_test[:10])

tensor([[178183.8906],
        [137942.5469],
        [146671.4062],
        [166703.4375],
        [177999.8906],
        [149255.2656],
        [163394.2969],
        [130299.7656],
        [177027.2500],
        [121461.8906]], device='cuda:0')


By applying the target normalization approach, we can now see that the model is demonstrating significant improvements in  **prediction diversity**. Moreover, we have an imporovement also in performance metrics:

- Without normalizing target variable we got:
    - Test loss (MSE): 4087500544.00, Test MAE: 52371.81

- While normalizing target variable we got:
    - Test loss (MSE): 2378938368.00, Test MAE: 37803.72